# calculating return values and periods from lmoments

In [2]:
import requests 
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime

import lmoments3 as lm
from lmoments3 import distr

import matplotlib.pyplot as plt
%matplotlib inline

## extracting time series from netcdf file

In [3]:
ds = xr.open_dataset("./data/pr_Amon_NorESM1-M_historical_r1i1p1_185001-200512.nc")
print(ds)

<xarray.Dataset>
Dimensions:    (time: 1872, bnds: 2, lat: 96, lon: 144)
Coordinates:
  * time       (time) object 1850-01-16 12:00:00 ... 2005-12-16 12:00:00
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    pr         (time, lat, lon) float32 ...
Attributes: (12/26)
    institution:            Norwegian Climate Centre
    institute_id:           NCC
    experiment_id:          historical
    source:                 NorESM1-M 2011  atmosphere: CAM-Oslo (CAM4-Oslo-n...
    model_id:               NorESM1-M
    forcing:                GHG, SA, Oz, Sl, Vl, BC, OC
    ...                     ...
    table_id:               Table Amon (27 April 2011) a5a1c518f52ae340313ba0...
    title:                  NorESM1-M mo

In [4]:
# converting precipitation to m/day units

ds["pr"].data = ds["pr"].data * 86.4
ds["pr"].attrs["units"] = 'm/day' 

In [7]:
# getting dataset specifically for defined location
# coordinates for berkeley, california

lat = 37.87
lon = 122.27
ds_berk = ds.sel(lon=lon, lat=lat, method='nearest')

In [11]:
# converting to timeseries dataframe

sr = ds_berk['pr'].to_pandas()
sr.index = sr.index.to_datetimeindex()
df = pd.DataFrame({'time':sr.index, 'pr':sr.values})

/var/folders/bz/l7frggbx71n9k19wxh7hvwsm0000gn/T/ipykernel_79472/1416090827.py:4: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  sr.index = sr.index.to_datetimeindex()


## using l-moments to calculate extreme value

In [13]:
# get annual max timeseries

annual_max = df.groupby("time")['pr'].max()
annual_max

time
1850-01-16 12:00:00    0.000177
1850-02-15 00:00:00    0.000602
1850-03-16 12:00:00    0.001685
1850-04-16 00:00:00    0.000859
1850-05-16 12:00:00    0.001908
                         ...   
2005-08-16 12:00:00    0.002018
2005-09-16 00:00:00    0.001186
2005-10-16 12:00:00    0.000800
2005-11-16 00:00:00    0.000338
2005-12-16 12:00:00    0.000643
Name: pr, Length: 1872, dtype: float32

In [15]:
# get parameters

lm_params = lm.lmom_ratios(annual_max, 5)

In [17]:
# fit to gev distribution

paras = distr.gev.lmom_fit(annual_max)
fitted_gev = distr.gev(**paras)

In [18]:
# calculate how to get the probability of a certain return value occuring

1-fitted_gev.cdf(0.00007)

0.9243327933011254